# EECS 738 Comment Toxicity Prediction Model

In [1]:
import sys
sys.path.insert(0, '../src/')
import tools

In [2]:
# establish file toolkit
t = tools.tools('../data/train.csv')

Processing training data...
Removing Punctuation...
Building Stop Word Dictionary...
About to get all good comments


../src/tools.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  comments_scores['comment_text'] = self.processCommentList(comments_scores['comment_text'])


In [3]:
import CommentPredictor as CP

In [4]:
mod = CP.CommentPredictor(t)

In [5]:
preds = mod.getPreds()

In [6]:
preds.head(5)

,target,comment_text
11,0.00,this is a great story man i wonder if the pe...
31,0.66,yet call out all muslims for the acts of a few...
51,0.00,because the people who drive cars more are the...
58,0.00,mormons have had a complicated relationship wi...
111,0.66,i'm doing the same thing


In [7]:
comms = t.comments_scores

In [8]:
comms.head(5)

,target,comment_text
11,0.4400,this is a great story man i wonder if the pe...
31,0.9125,yet call out all muslims for the acts of a few...
51,0.0000,because the people who drive cars more are the...
58,0.3000,mormons have had a complicated relationship wi...
111,0.0000,i'm doing the same thing
